# Merging C++ and *Python* Using *Cython*

The ability to augment your analysis code using selected components of a **pre-existing** software framework is extremely useful. It is **not** typically the case that implementation of a computational scientific analysis requires the use an **entirely** novel numerical algorithms. Even if software that performs the intended analysis does not exist, it is likely that elements of one or more software frameworks can be **combined** to synthesize the required functionality. 

* If **source code** written using the **same programming language** is available for the required software components, the most straightforward approach may entail **aggregating and compiling** the code that defines the required functionality. Having access to source code also enables implementation of **fine-grained modifications** to the numerical algorithm. 

* Instead of providing the entire source code, many modern software frameworks are distributed using a combination of **shared libraries** and **header files**. If header files are provided that are **compatible** with the programming language that is being used to implement a scientific analysis, then the functionality provided by the framewor can be straightforwardly incorporated.

* The **functional components** of shared library files are simply **binary representations** of **machine-level instructions** for manipulating input data. In principle, these instructions can be executed in multiple contexts, provided that a suitable **interface description** is available. A language-compatible header file is an example of such an interface description but other options are possible.

* Defining and implementing an appropriate interface can be a non-trivial process. Fortunately, many tools exist to streamline the process. The [**Cython**](http://cython.org/index.html) utility generates the required interface elements to **expose** the functionality of shared libraries written in the **C** programming language. The exposed functionality is accessible as elements of a _Python_ **module** that can be `import`ed into any _Python_ interpreter.

## Preliminaries
Generating _Python_ modules using **Cython** entails execution of shell commands. These commands have been assembled into a [**bash shell script**](http://localhost:8888/edit/commands.sh). The script accepts a single command line argument that determines the actions it performs.

## Part 1: A Trivial Cython Example
This example uses **Cython** utility to generates _Python_ module called `TrivialCython` that contains a single _Python_ `print` statement. Despite the simplicity of the `TrivialCython`, **Cython** requires **two** input files to generate the module.

1. [*`TrivialCython.pyx`*](http://localhost:8888/edit/TrivialCython.pyx) - A Cython _implementation_ file written using the _Python_-like Cython language.
2. [*`TrivialSetup.py`*](http://localhost:8888/edit/TrivialSetup.py) - A Cython _build script_ file written in Python.

To build the `TrivialCython` module **Cython** invoke the `_commands.sh_` using "`trivial`" as the command line argument i.e.

    $ sh commands.sh trivial

The generated module is stored as a **shared library** with the same base name as the module and the usual `.so` suffix. **Note** that the usual `lib` prefix is **not** prepended.

Importing the module implicitly invokes the single statement it contains.

In [ ]:
import TrivialCython
'''
Demonstrate that documentation defined in Cython files is
propagated to the Python module.
'''
help(TrivialCython)

## Part 2: A Simple Cython Example
This example uses **Cython** utility to generates _Python_ module called `SimpleCython`, which defines a simple function that generates a list of values separated by a caller-specified step size. 

Like `TrivialCython`, **Cython** requires **two** input files to generate the `SimpleCython` module.

1. [*`SimpleCython.pyx`*](http://localhost:8888/edit/SimpleCython.pyx) - A Cython _implementation_ file written using the _Python_-like Cython language.
2. [*`SimpleSetup.py`*](http://localhost:8888/edit/SimpleSetup.py) - A Cython _build script_ file written in Python.

The `SimpleCython.pyx` file contains code that is designed to initialize the generated module when it is used as a **standalone** _Python_ script. Recall that the `__name__` variable will be equal to `"__main__"` if this is the case.

To build the `SimpleCython` module **Cython** invoke the `_commands.sh_` using "`simple`" as the command line argument i.e.

    $ sh commands.sh simple

In [ ]:
import SimpleCython

Elements of the `SimpleCython` module can be references using the usual `.` operator.

In [ ]:
SimpleCython.SimpleFunction(5)

## Part 3: An Example Using Cython's Type-Specification Mechanism
This example uses **Cython** utility to generates _Python_ module called `TypedPython` that demonstrates the use of the **`cdef`** keyword that is defined by the **Cython** language.

Like `TrivialCython` and `SimpleCython`, **Cython** requires **two** input files to generate the `TypedCython` module.

1. [*`TypedCython.pyx`*](http://localhost:8888/edit/TypedCython.pyx) - A Cython _implementation_ file written using the _Python_-like Cython language.
2. [*`TypedSetup.py`*](http://localhost:8888/edit/TypedSetup.py) - A Cython _build script_ file written in Python.

To build the `TypedCython` module **Cython** invoke the `_commands.sh_` using "`typed`" as the command line argument i.e.

    $ sh commands.sh typed

In [ ]:
import TypedCython

## Part 4: C++ 

This section demonstrates how the **Cython** utility can be used to map the functionality provided by the `StatsCalculator` C++ class onto symbols of a _Python_ module called `StatsCalcCython`.

### Part 4.1: Defining an Interface of Functions with C Linkage

Recent releases of Cython (since version 0.13) support the definition of **direct** mappings between **C++** code and _Python_ module symbols ** without** the need to define an intermediate interface of functions with C linkage. See the section on [Using C++ in **Cython**](http://docs.cython.org/src/userguide/wrapping_CPlusPlus.html) from the [online Users' Guide for **Cython**](http://docs.cython.org/src/userguide/index.html) for more details. 

To **declare** an interface of C++ functions with C linkage, simply enclose the function declarations in a code block that is prepended with the token pair **`extern "C"`**. Note that the interface is defined using **functions** and **not** class methods. Header files that define interfaces with C linkage must typically be compiled using **_both_** **C** and **C++** compilers. The **`extern "C"`** token pair is **not** valid in the **C** language and the tokens that define the **`extern "C"`** code bock must be hidden from **C** compilers using **C Preprocessor conditional** structures that depend on the preprocessor macro **`__cplusplus`**. If the file is being processed by a **C++** compiler, then the **`__cplusplus`** macro will be **defined**. Otherwise **`__cplusplus`** will **not** be defined. Schematically, a **C++** header file that defines an interface of functions with C linkage might have the following layout. 

```C++
#ifdef __cplusplus
/* 
C++ entities with C++ linkage 
*/
extern "C" {
#endif 
/* function declarations with C linkage */
#ifdef __cplusplus
}
#endif
```

An appropriately modified version of the `StatsCalculator.h` header file has been placed at
[`cppSourceFiles/include/StatsCalculator.h`](http://localhost:8888/edit/cppSourceFiles/include/StatsCalculator.h)

The **definitions** of the interface functions may contain **C++**-specific (i.e. **C**-incompatible) code and should be compiled using a **C++** compiler. To prevent the compiler from generating mangled symbol names, the **`extern "C"`** token pair must be prepended to the opening clause of each function deifinition e.g.

```C++
extern "C" void functionWithCLinkage(){
    /* Function body (may include C++ code). */
}
```

The definitions of the interface functions have been placed in the
[`cppSourceFiles/src/StatsCalculatorCAPI.cpp`](http://localhost:8888/edit/cppSourceFiles/src/StatsCalculatorCAPI.cpp) file.

To generate a shared library that includes symbols for the interface that have C linkage, invoke 

    $ sh commands.sh buildcpp

### Part 4.2: Writing the Cython _Definition_ (`.pxd`) file

**Cython** **_definition_** files have the `.pxd` suffix and contain **declarations** of the interface functions written in the (_Python_-like) **Cython language**. 

The definitions should be enclosed within a Python-like code block with the opening clause

```Cython
cdef extern from "HeaderFile.h" :
    # Cython function definitions...
```

The **`extern`** keyword specifies that the interface functions **declared** in `HeaderFile.h` are **defined** in a separate **shared library**. Typically, Cython function declarations have identical syntax to their counterparts in the C++ header file.

 [`StatsCalcCython.pxd`](http://localhost:8888/edit/StatsCalcCython.pxd) is a **Cython** definition file that declares the interface functions for the `StatsCalculator` class.

### Part 4.3: Writing the Cython _Implementation_ (`.pyx`) file
**Cython** **_implementation_** files have the `.pyx` suffix and contain implementations of functions, and classes written in the Cython language. Declarations of functions from external shared libraries that are defined in a separate **Cython** definition file can be imported using the **`cimport`** directive i.e.

```Cython
cimport StatsCalcCython # Note that the suffix is not required
```

**Note:** In fact, if the definition file and the implementation file share the same base name (i.e. StatsCalcCython) , an explicit **`cimport`** directive is not required - the import will be performed automatically.

The **Cython** language defines three **keywords** that can be used to define functions, classes or class methods.

1. **`def`** behaves identically to its _Python_ equivalent. Entities that are defined using **`def`** **will** be available in the _Python_ module that is ultimately generated.
2. **`cdef`** defines entities that will **not** be available in the _Python_ module that is ultimately generated but can offer substantially enhanced performance because they are compiled as **C** functions and avoid the substantial computational overhead that is introduced by e.g. the _Python_ type inference system.
3. **`cpdef`** defines that **will** be available in the _Python_ module that is ultimately generated. The **`cpdef`** keyword induces **Cython** to generate two versions of the function. One version behaves as if it had been defined using **`cdef`** and provides high performance. The second behaves as if it had been defined using **`def`**. The variant that is invoked is context dependent. The high performance variant will be invoked if the function is called from within **`cdef`**-defined function, while the slower _Python_-identical variant is provided by the generated module and called from ordinary _Python_ contexts. 

[`StatsCalcCython.pyx`](http://localhost:8888/edit/StatsCalcCython.pyx) is a **Cython** implementation file that defines a class that wraps the interface functions for the `StatsCalculator` class. Note that the implementation need **not** simply provide a direct mapping between the C++ functions and _Python_ equivalents. It can add add additional functionality that leverages the high performance C-style functions.

### Part 4.4: Writing the Cython _Build Script_

**Cython** delegates many stages of the build process such as compiler invocations and source code generation to the **`distutils`** package, which is widely used by active _Python_ module developers.  

The **`Cython.Build.cythonize()`** function is the core of the **Cython** build system, which generates appropriately initialized instances of the **`distutils.extension.Extension`** class. 



Generation of the Python module is delegated to the **`distutils.core.setup(…)`** function, which uses the initialized **`Extension`** instance for configuration. The **`setup()`** function generates a file written in **C** containing boilerplate **C**→_Python_ "bridging" code, and uses that code to build a shared library that acts as a _Python_ module.

[`StatsCalcSetup.py`](http://localhost:8888/edit/StatsCalcSetup.py) is a commented **Cython** build script that explains the arguments that are required by each of the functions it invokes.

To build the `StatsCalcCython` module **Cython** invoke the `_commands.sh_` using "`statscalc`" as the command line argument i.e.

    $ sh commands.sh simple

## Part 4.5: Testing the `StatsCalcCython` Module

### Part 4.5.1: Import and Inspect `StatsCalcCython`
Begin by `import`ing the **Cython**-generated `StatsCalcCython` module. The **built-in** _Python_ `dir(...)` function can be used to inspect the `StatsCalcCython.StatsCalculator` class and its methods.

In [ ]:
import StatsCalcCython
dir(StatsCalcCython.StatsCalculator)

### Part 4.5.2: Testing the `StatsCalcCython.StatsCalculator` class

Begin by instantiating a `StatsCalcCython.StatsCalculator` object. The underlying C++ interface will instantiate a `StatsCalculator` C++ object and begin managing it using the unique integer "handle" that the C++ interface generates. The `StatsCalcCython.StatsCalculator` constructor prints the value of the handle as verification.

In [ ]:
sc = StatsCalcCython.StatsCalculator()

Verify that the `StatsCalcCython.StatsCalculator.readFile(...)` method behaves as expected. The [`testData/unsortedNumbers`](http://localhost:8888/edit/testData/unsortedNumbers) file contains an appropriately formatted list of numeric values.

In [ ]:
sc.readFile('testData/unsortedNumbers')

Verify the expected functionality of the `getSum()`, `getMean()`, and `getStdDev()` methods provided by the `StatsCalcCython.StatsCalculator` _Python_ class.

In [ ]:
scSum = sc.getSum()
scMean = sc.getMean()
scStdDev = sc.getStdDev()
print "Data from 'testData/unsortedNumbers':"
print "Sum =",scSum,"and is of type",type(scSum)
print "Mean =",scMean,"and is of type",type(scMean)
print "Standard Deviation =",scStdDev,"and is of type",type(scStdDev)

Verify that the `writeStats()` method generates the expected statistical summary file. 

In [ ]:
sc.writeStats('testData/computedStats')

# Open the statistical summary file and print its contents
computedStats = open('testData/computedStats', 'r')
print computedStats.read()
computedStats.close()

The functionality that was **directly** mapped from the **C++** `StatsCalculator` class has now been tested and verified. The remaining cells excercise the `StatsCalcCython.StatsCalculator.appendValues()` method that was introduced by the intermediate **Cython** implementation file `StatsCalcCython.pyx`.

Begin by importing the `numpy`, `scipy` and `matplotlib` packages as well as the `matplotlib.pyplot` module.

In [ ]:
import numpy
import scipy
import matplotlib
from matplotlib import pyplot
%matplotlib inline

Use the `scipy.stats.norm` _Random Variable_ class to generate 2000 random variates constistent with a Gaussian distribution, specifying a mean of **7** and a standard deviation of **2.5**. This choice of distribution will enable straightforward verification of the computations that are performed by `StatsCalculator`. 

In [ ]:
from scipy import stats
from scipy.stats import norm
# define a dict of Gaussian parameter values
gaussianParameters = {"mean" : 7, "stddev" : 2.5}
variates = scipy.stats.norm.rvs(gaussianParameters["mean"], gaussianParameters["stddev"], size=2000)   

Use the `matplotlib.pyplot.hist(...)` function to plot a 50 bin histogram of the generated data.

In [ ]:
histProperties = pyplot.hist(variates, 50)

Instantiate another `StatsCalcCython.StatsCalculator`. The handle returned by the underlying **C++** interface should be different to the previous instance.

In [ ]:
sc2 = StatsCalcCython.StatsCalculator()

Invoke the `appendValues(...)` function, passing the generated Gaussian random variates (of type `numpy.array` - recall that the generic argument type `object` was specified in `StatsCalcCython.pyx`) as the single required argument.

In [ ]:
sc2.appendValues(variates)

Finally, invoke the `getSum()`, `getMean()`, and `getStdDev()` methods provided by `StatsCalcCython.StatsCalculator` and compare the computed mean and standard deviation with those used to generate the data.

In [ ]:
sc2Sum = sc2.getSum()
sc2Mean = sc2.getMean()
sc2StdDev = sc2.getStdDev()
print "Data from",len(variates), "random Gaussian Variates:"
print "Sum =",sc2Sum,"and is of type",type(sc2Sum)
print "Mean =",sc2Mean,"and is of type",type(sc2Mean)
print "Difference from True Mean =",(sc2Mean - gaussianParameters["mean"])
print "Standard Deviation =",sc2StdDev,"and is of type",type(sc2StdDev)
print "Difference from True Standard Deviation =",(sc2StdDev - gaussianParameters["stddev"])